In [0]:
import re
import requests
import pandas as pd
import urllib.request
from newspaper import Config
from newspaper import Article
from textblob import TextBlob
from bs4 import BeautifulSoup
from google.colab import drive
from urllib.request import urlopen

#Initial page to get results
page = 0
final_list = []
#Date in format mm/dd/yyyy
initdate = "4/16/2020"
enddate = "4/17/2020"
query = "Data Science" 
limit = 4
#This header is to get the real results
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}    
#Next three lines work with the newspaper library, to ensure scrape the webpage
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
config = Config()
config.browser_user_agent = user_agent

#Iterate through all the results in the search 
for x in range(0, limit):
    #creates the url to get results from google
    url = "http://www.google.com/search?q=" + query + "&tbs=cdr:1,cd_min:" + initdate + ",cd_max:" + enddate + "&sa=N&biw=1845&bih=977&dpr=1&start=" + str(page)
    page = page + 10
    #Next three lines get all the html tags for that set of results
    source_code = requests.get(url, headers = headers)
    article_content = source_code.content
    soup = BeautifulSoup(article_content, 'html5lib')
    #This class has the link to urls
    body = soup.find_all('div', class_='dbsr')
    #If tags found
    if len(body) > 0:
        #Iterate through all the tags
        for desc in soup.find("div",{"class":"dbsr"}):
            #gets the actual url to webpage
            strurl = desc.get('href')
            if strurl is not None:
              #initializes newspaper library
              article = Article(strurl, config=config, language='en')
              #When can't get the text, throws an exception 
              try:
                  #Next four lines get the text of the article
                  article.download()
                  article.parse()
                  #article.nlp()
                  finaltext = article.text
                  #Delete spaces from blank results
                  finaltext = finaltext.strip()
                  #If finaltext is a string
                  if finaltext:
                      #Filter only results in english language
                      if(TextBlob(finaltext).detect_language() == "en"):
                        #Delete all special characters from string and add carriage return
                        final_list.append(re.sub(r'\W+', ' ', finaltext) + '\n')
              except:
                  pass

#Results are written to file 
with open('/content/gdrive/My Drive/Data.txt', 'w') as f:
  f.writelines(final_list)
  f.close()